<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100" align="right" /> 

# Cours TAL - Laboratoire 2<br/>*POS taggers* pour le français dans spaCy et NLTK

**Objectif**

Comparer l'étiqueteur morphosyntaxique français prêt-à-l'emploi de spaCy avec deux étiqueteurs entraînés, l'un dans spaCy et l'autre dans NLTK.

## 1. Installation et test de spaCy

La boîte à outils spaCy est une librairie Python *open source* pour le TAL, dédiée à un usage en production. Les documents suivants vous seront utiles :
* comment [installer](https://spacy.io/usage) spaCy
* comment [télécharger un modèle](https://spacy.io/usage/models) pour une langue donnée (on appelle ces modèles des *trained pipelines* car ils enchaînent plusieurs traitements)
* comment faire les [premiers pas](https://spacy.io/usage/spacy-101) dans l'utilisation de spaCy

Veuillez installer spaCy, puis la *pipeline* pour le français appelée `fr_core_news_sm`.  Si vous utilisez *conda*, installez spaCy dans l'environnement du cours TAL.

In [103]:
import spacy
nlp = spacy.load("fr_core_news_sm") # charge la pipeline
import tqdm # permet l'affichage d'une barre de progression

**1a.** Une pipeline effectue un ensemble de traitements d'un texte en lui ajoutant des annotations.  Les traitements effectués par la pipeline `fr_core_news_sm` sont [documentés ici](https://spacy.io/models/fr#fr_core_news_sm).  La liste des traitements d'une pipeline figure dans son attribut `.pipe_names`.  On peut activer ou désactiver un traitement T avec, respectivement, les méthodes `.disable_pipe(T)` et `.enable_pipe(T)` appliquées à la pipeline.

* Veuillez afficher les traitements disponibles dans la pipeline `fr_core_news_sm` chargée ci-dessus sous le nom de `nlp` .
* Veuillez désactiver tous les traitements sauf `tok2vec` et `morphologizer` (on fait cela pour accélerer le traitement).
* Vérifiez que la désactivation a bien fonctionné en affichant les traitements activés.

In [104]:
# Veuillez écrire votre code ici.
print('Avant ->', nlp.pipe_names)
for name in nlp.pipe_names:
    nlp.disable_pipe(name)

nlp.enable_pipe('tok2vec')
nlp.enable_pipe('morphologizer')
print('Après :) ->', nlp.pipe_names)

Avant -> ['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
Après :) -> ['tok2vec', 'morphologizer']


In [105]:
from spacy.lang.fr.examples import sentences

**1b.** L'objet `sentences` chargé ci-dessus contient une liste de phrases en français. 

* Veuillez afficher les deux premières phrases de `sentences`.
* Veuillez analyser chacune de ces deux phrases avec la pipeline `nlp` puis afficher chaque token et son POS tag.
    * indication : aidez-vous de la [documentation](https://spacy.io/models/fr#fr_core_news_sm) de `fr_core_news_sm`
    * consigne d'affichage : indiquer le tag entre crochets après chaque token, comme ceci : Les \[DET\] robots \[NOUN\] ...
    * note : la documentation détaillée du POS tagging dans spaCy est [disponible ici](https://spacy.io/usage/linguistic-features)
* Veuillez commenter la tokenisation et les POS tags observés : vous semblent-ils corrects pour les deux phrases ?

In [106]:
def print_analyzed_sentence(sentence):
    analyzed_sentence = nlp(sentence)
    for tok in analyzed_sentence:
        print('\t-', tok.text, tok.pos_)


In [107]:
# Veuillez écrire votre code et votre commentaire ici.
print(sentences[0:2])
for i in range(2):
    print('Phrase N°',i+1,':\n')
    print_analyzed_sentence(sentences[i])

['Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars', "Les voitures autonomes déplacent la responsabilité de l'assurance vers les constructeurs"]
Phrase N° 1 :

	- Apple NOUN
	- cherche NOUN
	- à ADP
	- acheter VERB
	- une DET
	- start NOUN
	- - NOUN
	- up ADJ
	- anglaise NOUN
	- pour ADP
	- 1 NUM
	- milliard NOUN
	- de ADP
	- dollars NOUN
Phrase N° 2 :

	- Les DET
	- voitures NOUN
	- autonomes ADJ
	- déplacent ADV
	- la DET
	- responsabilité NOUN
	- de ADP
	- l' DET
	- assurance NOUN
	- vers ADP
	- les DET
	- constructeurs NOUN


*Commentaires :*

La tokenisation est correcte dans l'ensemble mais on aurait préféré que le mot "start-up" de la première phrase ne soit pas divisés en 3 tokens différents. La tokenisation de la deuxième phrase semble correcte.

Concernant le tagging, il y a plusieurs problèmes :
- Phrase N°1 :
    - 'cherche' devrait être un verbe, pas un nom
    - étant donné que le mot 'start-up' a été divisé en plusieurs tokens, le tagging n'a plus trop de sens
    - 'anglaise' devrait être un adjectif
- Phrase N°2 :
  - 'déplacent' est un verbe conjugué, pas un adverbe

## 2. Prise en main des données

Les données sont fournies dans un format tabulaire dans l'archive `UD_French-GSD.zip` sur Cyberlearn.  Elles sont basées sur les données fournies par le projet [Universal Dependencies](https://github.com/UniversalDependencies/UD_French-GSD).  Leur format, appelé CoNLL-U, est [documenté ici](https://universaldependencies.org/format.html).  Veuillez placer les trois fichiers contenus dans l'archive dans un sous-dossier de ce notebook nommé `spacy_data`.

Les trois fichiers contiennent des phrases en français annotées avec les POS tags :
* le fichier `fr-ud-train.conllu` est destiné à l'entraînement
* le fichier `fr-ud-dev.conllu` est destiné aux tests préliminaires et aux réglages des paramètres
* le fichier `fr-ud-test.conllu` est destiné à l'évaluation finale.

**2a.** En inspectant les fichiers avec un éditeur texte, veuillez déterminer dans quelle colonne se trouvent les *tokens* des textes originaux, et dans quelle colonne se trouvent leurs étiquettes morpho-syntaxiques correctes (*POS tags*).  Que contient la troisième colonne ?

In [108]:
# Veuillez écrire vos réponses dans cette cellule.
# Colonne 0 : N° du token
# Colonne 1 : token
# Colonne 2 : lemme du token
# Colonne 3 : Universal POS Tag (selon la doc du format CoNLL-U)


**2b.** Veuillez convertir les trois fichiers de données en des fichiers binaires utilisables par spaCy, en utilisant la [commande 'convert' fournie par spaCy](https://spacy.io/api/cli#convert).  La commande est donnée ci-dessous, le premier dossier `./input_data` contient les 3 fichiers `.conllu` et le dossier `./spacy-data` contiendra les 3 résultats.

* Veuillez exécuter la commande de conversion.
* Combien de phrases environ (à 10 phrases près) contient chaque fichier (*train*, *dev*, *test*) ?  Observez la commande et son résultat pour répondre.

In [109]:
#!python -m spacy convert ./data ./spacy_data --converter conllu  --n-sents 10 --lang fr

In [110]:
# Veuillez indiquer les nombres de phrases ici.
# dev -> 148 * 10 phrases = 1480
# test -> 42 * 10 phrases = 420
# train -> 1456 * 10 phrases = 14560


**2c**. Les données des fichiers convertis peuvent être chargées dans un objet de type `DocBin`.  Dans notre cas, un tel objet contient un ensemble de documents, chacun contenant 10 phrases.  Chaque document est un objet de type `Doc`.  Le code donné ci-dessous vous permet de charger les données de test et vous montre comment les afficher.

* Veuillez stocker la première phrase des données de test dans une variable nommée `premiere_phrase_test`.
* Veuillez afficher cette phrase, ainsi que son type dans spaCy.

In [111]:
from spacy.tokens import DocBin
from spacy.tokens import Doc
test_data = DocBin().from_disk("./spacy_data/fr-ud-test.spacy")
# Exemple d'utilisation (afficher toutes les phrases)
# for doc in test_data.get_docs(nlp.vocab): 
#     for sent in doc.sents:
#         print(sent)

In [112]:
# Veuillez écrire votre code ici.
docs = test_data.get_docs(nlp.vocab)
first_document = next(docs)
sentences_first_document = first_document.sents
premiere_phrase_test = next(sentences_first_document)
print(premiere_phrase_test)


Je sens qu'entre ça et les films de médecins et scientifiques fous que nous avons déjà vus, nous pourrions emprunter un autre chemin pour l'origine.


## 3. Évaluation du POS tagger français de la pipeline `fr_core_news_sm`

**3a.** Veuillez effectuer le *POS tagging* avec spaCy de la `premiere_phrase_test` et afficher les résultats dans le format demandé au (1b).  Indication : convertissez la `premiere_phrase_test` dans un objet de type `Doc` en lui appliquant la méthode `.as_doc()`.  Cet objet peut être ensuite traité par la pipeline `nlp`.

In [113]:
# Veuillez écrire votre code ici.

doc_premiere_phrase_test = premiere_phrase_test.as_doc()

print_analyzed_sentence(doc_premiere_phrase_test)


	- Je PRON
	- sens VERB
	- qu' SCONJ
	- entre ADP
	- ça PRON
	- et CCONJ
	- les DET
	- films NOUN
	- de ADP
	- médecins NOUN
	- et CCONJ
	- scientifiques NOUN
	- fous PRON
	- que PRON
	- nous PRON
	- avons AUX
	- déjà ADV
	- vus VERB
	- , PUNCT
	- nous PRON
	- pourrions VERB
	- emprunter VERB
	- un DET
	- autre ADJ
	- chemin NOUN
	- pour ADP
	- l' DET
	- origine NOUN
	- . PUNCT


**3b.** Veuillez afficher les tags corrects de `premiere_phrase_test`, puis comparez-les visuellement les tags trouvés automatiquement au (3a).  Quelles différences trouvez-vous ?

In [114]:
# Veuillez écrire votre réponse ici.
comparison = []

for i, token in enumerate(doc_premiere_phrase_test):
    element = []
    element.append(token.text)
    element.append(token.pos_) # .pos_ -> prédiction
    element.append(token.tag_) # .tag_ -> valeur de référence (provenant du fichier)
    comparison.append(element)

# Comparaison complète
print("Comparaison complète : ")
for elem in comparison:
    print(elem[0], ' -----> Prédit : ', elem[1], '| Selon fichier : ', elem[2])

print("Différences uniquement : ")
for elem in comparison:
    if(elem[1] != elem[2]):
        print(elem[0], ' -----> Prédit : ', elem[1], '| Selon fichier : ', elem[2])



Comparaison complète : 
Je  -----> Prédit :  PRON | Selon fichier :  PRON
sens  -----> Prédit :  VERB | Selon fichier :  VERB
qu'  -----> Prédit :  SCONJ | Selon fichier :  SCONJ
entre  -----> Prédit :  ADP | Selon fichier :  ADP
ça  -----> Prédit :  PRON | Selon fichier :  PRON
et  -----> Prédit :  CCONJ | Selon fichier :  CCONJ
les  -----> Prédit :  DET | Selon fichier :  DET
films  -----> Prédit :  NOUN | Selon fichier :  NOUN
de  -----> Prédit :  ADP | Selon fichier :  ADP
médecins  -----> Prédit :  NOUN | Selon fichier :  NOUN
et  -----> Prédit :  CCONJ | Selon fichier :  CCONJ
scientifiques  -----> Prédit :  NOUN | Selon fichier :  NOUN
fous  -----> Prédit :  PRON | Selon fichier :  ADJ
que  -----> Prédit :  PRON | Selon fichier :  PRON
nous  -----> Prédit :  PRON | Selon fichier :  PRON
avons  -----> Prédit :  AUX | Selon fichier :  AUX
déjà  -----> Prédit :  ADV | Selon fichier :  ADV
vus  -----> Prédit :  VERB | Selon fichier :  VERB
,  -----> Prédit :  PUNCT | Selon fichier :

*Réponse :* Le seul tag différent pour la première phrase est pour le mot "fous". C'est un adjectif, mais nlp l'a classifié comme un pronom (choix un peu étrange...).

In [115]:
from spacy.scorer import Scorer
from spacy.training import Example

In [116]:
scorer = Scorer()

**3c.** Au lieu de compter manuellement combien de tags sont différents entre la référence et le résultat de la pipeline `nlp`, vous allez utiliser la classe `Scorer` de spaCy.  Une instance de cette classe permet de calculer les scores d'une liste d'objets de type `Exemple`, en fonction des annotations disponibles dans les objets.  Un objet de type `Exemple` contient deux objets de type `Doc`, l'un avec les annotations correctes et l'autre avec les annotations produites par une pipeline.  La [documentation de la méthode](https://spacy.io/api/scorer#score) `Scorer.score(..)` vous sera utile. 

* Veuillez calculer la justesse (*accuracy*) du *POS tagging* de `premiere_phrase_test`. 
* Veuillez justifier la valeur du score obtenu en utilisant votre réponse du (3b).

In [117]:
# Veuillez écrire votre code ici.
docPos = [tok.pos_ for tok in doc_premiere_phrase_test]

example = Example(nlp(premiere_phrase_test.as_doc()), premiere_phrase_test.as_doc())

print('Accuracy :', scorer.score([example])['tag_acc'])

print('Vérification :', 1 - 1/len(doc_premiere_phrase_test))


Accuracy : 1.0
Vérification : 0.9655172413793104


**3d.** Veuillez calculer la précision du *POS tagging* de la pipeline `nlp` sur toutes les données de test présentes dans `test_data`.  Comment se compare le score obtenu avec celui mentionné [dans la documentation](https://spacy.io/models/fr#fr_core_news_sm) du modèle `fr_core_news_sm` ?

In [118]:
# Veuillez écrire votre code ici, suivi de votre réponse à la question.

scorer = Scorer()

examples = []
for doc in test_data.get_docs(nlp.vocab):
    for sent in doc.sents:
        example = Example(nlp(sent.as_doc()), sent.as_doc())
        examples.append(example)
print('Accuracy :', scorer.score(examples)['pos_acc'])


Accuracy : 0.9172654690618762


*Réponse :* Nous n'obtenons pas exactement le même score (0.91 au lieu de 0.96 dans la doc). C'est un petite différence donc on peut en conclure que l'ensemble de test est plus représentatif.

## 4. Entraîner puis évaluer un nouveau POS tagger français dans spaCy

Le but de cette partie est d'entraîner une pipeline spaCy pour le français sur les données de `fr-ud-train.conllu`, puis de comparer le modèle obtenu avec le modèle prêt-à-l'emploi testé au point précédent.  Les [instructions d'entraînement](https://spacy.io/usage/training#quickstart) de spaCy vous montrent comment entraîner une pipeline avec un POS tagger.

**4a.** Paramétrage de l'entraînement :
* générez un fichier de départ grâce à [l'interface web](https://spacy.io/usage/training#quickstart), en indiquant que vous voulez seulement un POS tagger dans la pipeline ;
* sauvegardez le code généré par spaCy dans un fichier local `base_config.cfg` ;
* générez un fichier `config.cfg` sur votre ordinateur en exécutant la ligne de commande suivante. 

In [130]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Enfin, veuillez effectuer l'entraînement avec la ligne de commande suivante.  Faites plusieurs essais, d'abord avec un petit nombre d'époques, pour estimer le temps nécessaire et observer les messages affichés.  Puis augmentez progressivement le nombre d'époques.  Quel est le critère qui vous permet de décider que vous avez un nombre suffisant d'époques ?  Dans quel dossier se trouve le meilleur modèle ?

In [131]:
!python -m spacy train config.cfg \
  --output ./myPOStagger1 \
  --paths.train ./spacy_data/fr-ud-train.spacy \
  --paths.dev ./spacy_data/fr-ud-dev.spacy \
  --verbose

✔ Created output directory: myPOStagger1
ℹ Saving to output directory: myPOStagger1
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'tagger']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS TAGGER  TAG_ACC  SCORE 
---  ------  ------------  -----------  -------  ------
  0       0          0.00       211.77    36.34    0.36
  0     200        315.96     10403.17    90.36    0.90
  0     400        287.45      4496.17    91.62    0.92
  0     600        223.10      3449.18    92.17    0.92
  0     800        216.83      3458.35    92.55    0.93
  0    1000        192.05      3030.81    92.62    0.93
  0    1200        185.09      2922.23    92.96    0.93
  0    1400        172.84      2749.04    92.98    0.93
  1    1600        147.28      2230.07    93.11    0.93
  1    1800        138.32      2029.00    93.

[2025-03-13 14:39:49,083] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
[2025-03-13 14:39:54,855] [INFO] Set up nlp object from config
[2025-03-13 14:39:54,873] [DEBUG] Loading corpus from path: spacy_data\fr-ud-dev.spacy
[2025-03-13 14:39:54,878] [DEBUG] Loading corpus from path: spacy_data\fr-ud-train.spacy
[2025-03-13 14:39:54,878] [INFO] Pipeline: ['tok2vec', 'tagger']
[2025-03-13 14:39:54,881] [INFO] Created vocabulary
[2025-03-13 14:39:54,881] [INFO] Finished initializing nlp object
[2025-03-13 14:40:17,342] [INFO] Initialized pipeline components: ['tok2vec', 'tagger']
[2025-03-13 14:40:17,359] [DEBUG] Loading corpus from path: spacy_data\fr-ud-dev.spacy
[2025-03-13 14:40:17,361] [DEBUG] Loading corpus from path: spacy_data\fr-ud-train.spacy


*Réponse :*

Le modèle s'est entraîné pendant 5 epochs. Dans l'interface permettant de générer le fichier de config, nous avons sélectionné "efficiency" comme optimisation. C'est donc sûrement la métrique (`TAG_ACC`) qui est utilisée comme critère d'arrêt. Le meilleur modèle se trouve dans `myPOStagger1/model-best/`.

**4b.**  Veuillez charger le meilleur modèle (pipeline) dans la variable `nlp2` et afficher la *POS tagging accuracy* sur le corpus de test.  Le composant de la pipeline étant un *POS tagger*, vous devrez évaluer la propriété *tag_acc*. 

In [ ]:
# Veuillez écrire votre code ici.
nlp2 = spacy.load('myPOStagger1/model-best/')


In [134]:
for doc in test_data.get_docs(nlp2.vocab):
    for sent in doc.sents:
        example = Example(nlp2(sent.as_doc()), sent.as_doc())
        examples.append(example)
print('Accuracy :', scorer.score(examples)['pos_acc'])

Accuracy : 0.9586327345309381


*Commentaire :*

L'accuracy est légèrement meilleure : ~0.96 au lieu de ~0.92.

## 5. Entraîner puis évaluer un POS tagger pour le français dans NLTK

Le but de cette partie est d'utiliser le POS tagger appelé *Averaged Perceptron* fourni par NLTK, en l'entraînant pour le français sur les mêmes données que ci-dessus, importées cette fois-ci avec NLTK.  Pour une introduction au POS tagging avec NLTK, voir le [Chapitre 5.1 du livre NLTK](http://www.nltk.org/book/ch05.html).

Remarques :
* pour l'anglais, des taggers pré-entraînés sont disponibles dans NLTK ;
* pour appliquer un tagger existant, on écrit `nltk.pos_tag(sentence)` où `sentence` est une liste de tokens et on obtient des paires (token, TAG) ;
* l'implémentation de *Averaged Perceptron* a été faite par [Mathew Honnibal de Explosion.AI](https://explosion.ai/blog/part-of-speech-pos-tagger-in-python), la société qui a créé spaCy.

**5a.** Veuillez charger les données d'entraînement et celles de test grâce à la classe `ConllCorpusReader` de NLTK.  [La documentation de cette classe](https://www.nltk.org/api/nltk.corpus.reader.conll.html#nltk.corpus.reader.conll.ConllCorpusReader) vous montrera comment indiquer les colonnes qui contiennent les tokens ('words') et les tags corrects ('pos').  Une fois les données chargées dans une variable, vous pouvez accéder aux phrases et aux tags avec la méthode `.tagged_sents()`.

In [194]:
from nltk.corpus.reader.conll import ConllCorpusReader

In [196]:
# Veuillez écrire votre code ici.

corpus_train = ConllCorpusReader(root='./data', fileids='fr-ud-train.conllu', columntypes=['ignore', 'words', 'ignore', 'pos'], encoding='utf8')
corpus_test = ConllCorpusReader(root='./data', fileids='fr-ud-test.conllu', columntypes=['ignore', 'words', 'ignore', 'pos'], encoding='utf8')


**5b.** Pour entraîner un POS tagger du type Averaged Perceptron, vous utiliserez le sous-module `nltk.tag.perceptron` du [module NLTK contenant les taggers](http://www.nltk.org/api/nltk.tag.html).  Les fonctions d'entraînement et de test sont documentées dans ce module.  Après l'entraînement, le réseau de neurones est enregistré dans un fichier `.pickle`, qui est écrasé à chaque entraînement si vous n'en faites pas une copie.  On peut également lire un fichier `.pickle` dans un tagger.

Veuillez écrire le code pour entraîner le POS tagger sur les données d'entraînement.  Comme au (4), pensez augmenter graduellement le nombre d'époques (appelées 'itérations' dans NLTK).

Combien de temps prend l'entraînement ?  Quelle est la taille du fichier enregistré ?

In [197]:
import os
# import nltk
# nltk.download('averaged_perceptron_tagger') # à exécuter la première fois
from nltk.tag.perceptron import PerceptronTagger

In [198]:
ptagger = PerceptronTagger(load=False)

In [ ]:
from nltk import pos_tag

sentences = corpus_train.tagged_sents()

# todo : trouver l'erreur ici
for sent in sentences:
    print(sent)

ptagger.train(sentences=sentences, nr_iter=1)


[('Les', 'DET'), ('commotions', 'NOUN'), ('cérébrales', 'ADJ'), ('sont', 'AUX'), ('devenu', 'VERB'), ('si', 'ADV'), ('courantes', 'ADJ'), ('dans', 'ADP'), ('ce', 'DET'), ('sport', 'NOUN'), ("qu'", 'SCONJ'), ('on', 'PRON'), ('les', 'PRON'), ('considére', 'VERB'), ('presque', 'ADV'), ('comme', 'ADP'), ('la', 'DET'), ('routine', 'NOUN'), ('.', 'PUNCT')]
[("L'", 'DET'), ('œuvre', 'NOUN'), ('est', 'AUX'), ('située', 'VERB'), ('dans', 'ADP'), ('la', 'DET'), ('galerie', 'NOUN'), ('des', '_'), ('de', 'ADP'), ('les', 'DET'), ('batailles', 'NOUN'), (',', 'PUNCT'), ('dans', 'ADP'), ('le', 'DET'), ('château', 'NOUN'), ('de', 'ADP'), ('Versailles', 'PROPN'), ('.', 'PUNCT')]
[('Le', 'DET'), ('comportement', 'NOUN'), ('de', 'ADP'), ('la', 'DET'), ('Turquie', 'PROPN'), ('vis-à-vis', 'NOUN'), ('du', '_'), ('de', 'ADP'), ('le', 'DET'), ('problème', 'NOUN'), ('palestinien', 'ADJ'), ('a', 'AUX'), ('fait', 'VERB'), ("qu'", 'SCONJ'), ('elle', 'PRON'), ("n'", 'ADV'), ('est', 'VERB'), ('plus', 'ADV'), ('en', 

ValueError: Inconsistent number of columns:
1	À	à	ADP	_	_	6	case	_	_
2	partir	partir	VERB	_	VerbForm=Inf	1	fixed	_	_
3-4	du	_	_	_	_	_	_	_	_
3	de	de	ADP	_	_	1	fixed	_	_
4	le	le	DET	_	Definite=Def|Gender=Masc|Number=Sing|PronType=Art	6	det	_	_
5	XXIe	XXIe	ADJ	_	Gender=Masc|Number=Sing|NumType=Ord	6	amod	_	_
6	siècle	siècle	NOUN	_	Gender=Masc|Number=Sing	19	obl	_	SpaceAfter=No
7	,	,	PUNCT	_	_	19	punct	_	_
8	les	le	DET	_	Definite=Def|Gender=Masc|Number=Plur|PronType=Art	9	det	_	_
9	recensements	recensement	NOUN	_	Gender=Masc|Number=Plur	19	nsubj	_	_
10	réels	réel	ADJ	_	Gender=Masc|Number=Plur	9	amod	_	_
11-12	des	_	_	_	_	_	_	_	_
11	de	de	ADP	_	_	13	case	_	_
12	les	le	DET	_	Definite=Def|Gender=Fem|Number=Plur|PronType=Art	13	det	_	_
13	communes	commune	NOUN	_	Gender=Fem|Number=Plur	9	nmod	_	_
14	de	de	ADP	_	_	18	case	_	_
15	moins	moins	ADV	_	_	18	det	_	_
16	de	de	ADP	_	_	15	fixed	_	_
17	10 000	10 000	NUM	_	_	18	nummod	_	_
18	habitants	habitant	NOUN	_	Gender=Masc|Number=Pl

In [ ]:
# Veuillez écrire ici vos réponses aux questions (temps d'entraînement et taille du modèle).


**5c.** Veuillez évaluer le tagger sur les données de test et afficher le taux de correction.

In [ ]:
# Veuillez écrire votre code ici.


## 6. Conclusion

Veuillez remplir le tableau suivant avec les scores obtenus et discuter brièvement comment se comparent les trois POS taggers sur ces données de test.

| spaCy (partie 3) | spaCy (partie 4) | NLTK (partie 5) | 
|------------------|------------------|-----------------|
| tagger fourni    | tagger entraîné  | tagger entraîné |
| 0.917              | 0.958              | pas fait             |


**Fin du Labo.** Veuillez nettoyer ce notebook en gardant seulement les résultats désirés, l'enregistrer, et le soumettre comme devoir sur Cyberlearn.